# Simulación de una avenida con paso peatonal
En esta simulación se representa una avenida con dos carriles y un paso peatonal. El tráfico se controla con dos semáforos que detienen a los autos en el paso peatonal.

## Librerías para la simulación

In [ ]:
pip install agentpy

In [ ]:
pip install matplotlib==3.4.3

In [ ]:
import agentpy as ap
import numpy as np
import matplotlib.pyplot as plt
import math
import IPython

## Definición del agente semáforo

In [ ]:
class Semaphore(ap.Agent):
    """ 
        Esta clase define a un semáforo.
    """

    def setup(self):
        """ Este método se utiliza para inicializar al semáforo. """
        
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación

        self.direction = [0, 1]      # Dirección a la que apunta el semáforo

        self.state = 0               # Estado del semáforo 0 = verde, 1 = amarillo, 2 = rojo
        self.state_time = 0          # Tiempo que ha durado el semáforo en el estado actual

        self.green_duration = 50     # Tiempo que dura el semáforo en verde
        self.yellow_duration = 5     # Tiempo que dura el semáforo en amarillo
        self.red_duration = 45       # Tiempo que dura el semáforo en rojo        

    def update(self):
        """ Este método actualiza el estado del semáforo. """
        self.state_time += self.step_time

        if self.state == 0:
            # Caso en el que el semáforo está en verde
            if self.state_time >= self.green_duration:
                self.state = 1
                self.state_time = 0
        elif self.state == 1:
            # Caso en el que el semáforo está en amarillo
            if self.state_time >= self.yellow_duration:
                self.state = 2
                self.state_time = 0
        elif self.state == 2:
            # Caso en el que el semáforo está en rojo
            if self.state_time >= self.red_duration:
                self.state = 0
                self.state_time = 0

    def set_green(self):
        """ Este método forza el semáforo a estar en verde. """        
        self.state = 0
        self.state_time = 0

    def set_yellow(self):
        """ Este método forza el semáforo a estar en amarillo. """        
        self.state = 1
        self.state_time = 0

    def set_red(self):
        """ Este método forza el semáforo a estar en rojo. """        
        self.state = 2
        self.state_time = 0


## Definición del agente auto

In [ ]:
class Car(ap.Agent):
    """ 
        Esta clase define a un auto.
    """

    def setup(self):
        """ Este método se utiliza para inicializar un robot limpiador. """
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación

        self.direction = [1, 0]      # Dirección a la que viaja el auto
        self.speed = 0.0             # Velocidad en metros por segundo
        self.max_speed = 10          # Máxima velocidad en metros por segundo
        self.state = 1               # Car state: 1 = ok, 0 = dead

    def update_position(self):
        """ Este método se utiliza para inicializar la posición del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return

         # Actualiza la posición según la velocidad actual
        self.model.avenue.move_by(self, [self.speed*self.direction[0], self.speed*self.direction[1]])

    def update_speed(self):
        
        """ Este método se utiliza para inicializar la velocidad del auto. """

        # Verifica si el auto no ha chocado
        if self.state == 0:
            return
        
        # Obten la distancia más pequeña a uno de los autos que vaya en la misma dirección        
        p = self.model.avenue.positions[self]

        min_car_distance = 1000000
        for car in self.model.cars:
            if car != self:
                # Verifica si el carro va en la misma dirección
                dot_p1 = self.direction[0]*car.direction[0] + self.direction[1]*car.direction[1]                
                
                # Verifica si el carro está atrás o adelante
                p2 = self.model.avenue.positions[car]
                dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

                if dot_p1 > 0 and dot_p2 > 0:                     
                    d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)                    
                    
                    if min_car_distance > d:
                        min_car_distance = d
       
        # Obten la distancia al próximo semáforo
        min_semaphore_distance = 1000000
        semaphore_state = 0
        close_semaphore = self.model.semaphores[0]
        for semaphore in self.model.semaphores:

            # Verifica si el semáforo apunta hacia el vehículo
            dot_p1 = semaphore.direction[0]*self.direction[0] + semaphore.direction[1]*self.direction[1]
            
            # Verifica si el semáforo está adelante o atrás del vehículo
            p2 = self.model.avenue.positions[semaphore]
            dot_p2 = (p2[0]-p[0])*self.direction[0] + (p2[1]-p[1])*self.direction[1]

            if dot_p1 < 0 and dot_p2 > 0:                            
                d = math.sqrt((p[0]-p2[0])**2 + (p[1]-p2[1])**2)  
                
                if min_semaphore_distance > d:
                    close_semaphore = semaphore
                    min_semaphore_distance = d
                    semaphore_state = semaphore.state
        
        # Actualiza la velocidad del auto

        #print("Car " + str(self.id) + " has a " + str(min_semaphore_distance) + " distance with semaphore " + str(close_semaphore.id) + " with state " + str(semaphore_state))
      
        if min_car_distance < 2:
            self.speed = 0
            self.state = 0

        elif min_car_distance < 10:
              self.speed = np.maximum(self.speed - 200*self.step_time, 0)

        elif min_car_distance < 15:
              self.speed = np.maximum(self.speed - 80*self.step_time, 0)
                
        elif min_semaphore_distance < 15 and semaphore_state == 1:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)

        elif min_semaphore_distance < 40 and semaphore_state == 1:
            self.speed = np.maximum(self.speed - 5*self.step_time, 0)
            
        elif min_semaphore_distance < 45 and semaphore_state == 2:
            self.speed = np.maximum(self.speed - 20*self.step_time, 0)

        else:
            self.speed = np.minimum(self.speed + 5*self.step_time, self.max_speed)
                    


## Definición del modelo de la avenida

In [ ]:
class AvenueModel(ap.Model):
    """ Esta clase define un modelo para una avenida simple con semáforo peatonal. """

    def setup(self):
        """ Este método se utiliza para inicializar la avenida con varios autos y semáforos. """
        
        self.strFile = """{
                    "time": 10,
                    "fps": 20,
                    "cars": [
                        """

        self.frames = ""

        # Inicializa los agentes los autos y los semáforos        
        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.step_time =  self.p.step_time
        
        c_north = int(self.p.cars/2)
        c_south = self.p.cars - c_north

        for k in range(c_north):
            self.cars[k].direction = [0,1]

        for k in range(c_south):
            self.cars[k+c_north].direction = [0,-1]

        self.semaphores = ap.AgentList(self,2, Semaphore)
        self.semaphores.step_time =  self.p.step_time
        self.semaphores.green_duration = self.p.green
        self.semaphores.yellow_duration = self.p.yellow
        self.semaphores.red_duration = self.p.red
        self.semaphores[0].direction = [0, 1]
        self.semaphores[1].direction = [0, -1]

        # Inicializa el entorno
        self.avenue = ap.Space(self, shape=[self.p.size, self.p.size], torus = True)
                
        # Agrega los semáforos al entorno
        self.avenue.add_agents(self.semaphores, random=True)
        self.avenue.move_to(self.semaphores[0], [self.p.size*0.5 - 20, self.p.size*0.5 - 5])
        self.avenue.move_to(self.semaphores[1], [self.p.size*0.5 + 20, self.p.size*0.5 + 5])

        self.semaphores[0].state = 2
        
        # Agrega los autos al entorno
        self.avenue.add_agents(self.cars, random=True)
        for k in range(c_north):
            car = self.cars[k]
            x = self.p.size*0.5 + 10
            y = self.p.size*0.5 - 10 * (k+1)
            self.avenue.move_to(car, [x,y])

            self.initializeCar(car)
            self.strFile += ","
        
        for k in range(c_south):
            car = self.cars[k+c_north]
            x = self.p.size*0.5 - 10
            y = self.p.size*0.5 + 10 * (k+1)
            self.avenue.move_to(car, [x,y])

            self.initializeCar(car)
            if k < c_south-1:
              self.strFile += ","

        self.finishCars()
        self.initializeSemaphore(self.semaphores[0])
        self.strFile += ","
        self.initializeSemaphore(self.semaphores[1])
        self.finishInitialization()

    def step(self):
        """ Este método se invoca para actualizar el estado de la avenida. """        
        self.semaphores.update()

        self.cars.update_position()
        self.cars.update_speed()

        self.addFrame()
        if self.t < self.p.steps:
          self.frames += ","
    
    def update(self):
        if self.t >= self.p.steps:
            self.stop()
            
    def end(self):        
        self.closeFile()
    
    def transformDir(self, direction):
        if direction == [1, 0]:
            dir = 0
        elif direction == [0, 1]:
            dir = 90
        elif direction == [-1, 0]:
            dir = 180
        elif direction == [0, -1]:
            dir = 270
        return dir

    # Funciones para crear el archivo JSON

    # Agregar Auto a la inicialización del archivo
    def initializeCar(self, car):
        self.strFile += """{
                  "id": """ + str(car.id) + """,
                  "type": "CAR",
                  "origin": {
                      "x": """ + str(self.avenue.positions[car][1]-self.p.size*0.5) + """,
                      "z": """ + str(self.avenue.positions[car][0]-self.p.size*0.5) + """
                  },
                  "dir": """ + str(self.transformDir(car.direction)) + """
              }"""
    
    # Cerrar arreglo de autos e inicializar el de Semáforos
    def finishCars(self):
        self.strFile += """],
                "trafficLights": ["""

    # Inicializar semáforo en el archivo
    def initializeSemaphore(self, light):
        self.strFile += """{
                  "id": """ + str(light.id) + """,
                  "type": "LIGHT",
                  "origin": {
                      "x": """ + str(self.avenue.positions[light][1]-self.p.size*0.5) + """,
                      "z": """ + str(self.avenue.positions[light][0]-self.p.size*0.5) + """
                  },
                  "state": """ + str(light.state) + """,
                  "dir": """ + str(self.transformDir(light.direction)) + """
              }"""

    # Cerrar arreglo de semáforos   
    def finishInitialization(self):
        self.strFile += """],
              """
        self.frames = """"frames": [
            """
    # Agregar frame nuevo
    def addFrame(self):
        
        self.frames += """{
                            "frame": """ + str(self.t) + """,
                            "trafficLights": ["""
        carSize = len(self.cars)
        trafficSize = len(self.semaphores)

        for i in range(trafficSize): 
            self.frames += """{
                                  "id": """ + str(self.semaphores[i].id) + """,
                                  "state": """ + str(self.semaphores[i].state) + """
                              }"""
            if i < trafficSize-1:
              self.frames += ","
        

        self.frames += """],
                          "cars": ["""
        for i in range(carSize):
            self.frames += """{
                                  "id": """ + str(self.cars[i].id) + """,
                                  "x": """ + str(self.avenue.positions[self.cars[i]][1]-self.p.size*0.5) + """,
                                  "z": """ + str(self.avenue.positions[self.cars[i]][0]-self.p.size*0.5) + """,
                                  "dir": """ + str(self.transformDir(self.cars[i].direction)) + """
                              }"""
            if i < carSize-1:
              self.frames += ","
        
        self.frames += """]
            }"""

    # Cerrar el archivo y agregar el string de los objetos JSON
    def closeFile(self):
        with open("/content/drive/Shareddrives/Graficas/Reto/testData.json", "w") as file:
          self.strFile += self.frames + "]}"
          # Parse
          file.write(self.strFile)


## Crear Archivo JSON

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Funciones para visualización

In [ ]:
def animation_plot_single(m, ax):    
    ax.set_title(f"Avenida t={m.t*m.p.step_time:.2f}")
    
    colors = ["green", "yellow", "red"]
    
    pos_s1 = m.avenue.positions[m.semaphores[0]]    
    ax.scatter(*pos_s1, s=20, c=colors[m.semaphores[0].state])
    
    pos_s2 = m.avenue.positions[m.semaphores[1]]    
    ax.scatter(*pos_s2, s=20, c=colors[m.semaphores[1].state])
    
    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])    
    
    for car in m.cars:
        pos_c = m.avenue.positions[car]    
        ax.scatter(*pos_c, s=20, c="black")
    
    ax.set_axis_off()
    ax.set_aspect('equal', 'box')
        
def animation_plot(m, p):    
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)
    animation = ap.animate(m(p), fig, ax, animation_plot_single)
    return IPython.display.HTML(animation.to_jshtml(fps=20)) 

## Parámetros de la simulación

In [ ]:
parameters = {
    'step_time': 0.1,    # Procentaje de área cubierta por árboles
    'size': 1000,        # Tamaño en metros de la avenida
    'green': 15,          # Duración de la luz verde
    'yellow': 5,         # Duración de la luz amarilla
    'red': 10,           # Duración de la luz roja
    'cars': 10,          # Número de autos en la simulación
    'steps': 500,       # Número de pasos de la simulación
}

## Simulación de una corrida

In [ ]:
model = AvenueModel(parameters)
results = model.run()

Completed: 500 steps
Run time: 0:00:01.765024
Simulation finished


## Visualización de una corrida

In [ ]:
animation_plot(AvenueModel, parameters)

NameError: ignored